In [7102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7103]:
import re
from lxml import etree
import html as html_
!pip install titlecase
from titlecase import titlecase
!pip install googletrans
from googletrans import Translator
translator = Translator()
import csv
import json
import datetime
from xml.etree import ElementTree as ET

In [0]:
# AgrellA-Domd.xml                                    LefflerAC-HurManGorGodt.xml       StrindbergA-InforDoden.xml
# AgrellA-EnHufvudsak.xml                             LefflerAC-MosterMalvina.xml       StrindbergA-Kamraterna.xml
# AgrellA-EnLektion.xml                               LefflerAC-SannaKvinnor.xml        StrindbergA-Lycko-PersResa.xml
# AgrellA-Ensam.xml                                   LefflerAC-Skadespelerskan.xml     StrindbergA-Marodorer.xml
# AgrellA-Hvarfor.xml                                 LefflerAC-UnderToffeln.xml        StrindbergA-MasterOlof.xml
# AgrellA-IngridEnDodsKarlekssaga.xml                 LindheW-Modrar.xml                StrindbergA-Moderskarlek.xml
# AgrellA-Smastadslif.xml                             LundbergE-ForlatMig.xml           StrindbergA-Paria.xml
# AureliusH-FarmorsFodelsedag.xml                     MallingM-FruLeonora.xml           StrindbergA-Samum.xml
# BarthelsonA-Efterspel.xml                           MarholmL-Otteringning.xml         StrindbergA-TillDamaskus.xml
# BenedictssonV-Final.xml                             MeyersonG-DenNyaKlassen.xml       TopeliusT-ISmaMaffarnasLand.xml
# BenedictssonV-ITelefon.xml                          MeyersonG-EttPojkstreck.xml       TopeliusT-JeppeOchMurra.xml
# BenedictssonV-RomeosJulia.xml                       MolanderH-Varflod.xml             TopeliusZ-Askungen.xml
# BremerF-KonstnarnsFortviflan.xml                    StrindbergA-AnnoFyrtiatta.xml     TopeliusZ-JennysFormaningar.xml
# GeijerstamG-LarsAndersOchJanAndersOchDerasBarn.xml  StrindbergA-Bandet.xml            TopeliusZ-Krypskyttarne.xml
# HedbergF-Rospiggarna.xml                            StrindbergA-DenStarkare.xml       TopeliusZ-LuciasZiffror.xml
# IndebetouH-IDetGrona.xml                            StrindbergA-Fadren.xml            TopeliusZ-Perdita.xml
# IndebetouH-IFruntimmersveckan.xml                   StrindbergA-Fordringsegare.xml    TopeliusZ-Skogskonungen.xml
# KullgrenL-Karlek.xml                                StrindbergA-ForstaVarningen.xml   TopeliusZ-Snurran.xml
# KuylenstiernaE-NarNyarKom.xml                       StrindbergA-FrokenJulie.xml       TopeliusZ-StationSylvester.xml
# KuylenstiernaE-NuArDetJulIgen.xml                   StrindbergA-GilletsHemlighet.xml  TopeliusZ-TidernasSpegel.xml
# LefflerAC-Elfvan.xml                                StrindbergA-HerrBengtsHustru.xml  WahlenbergA-PaVakt.xml
# LefflerAC-Familjelycka.xml                          StrindbergA-Hostslask.xml         WahlenbergA-TvaValsprak.xml

In [0]:
with open('/content/drive/My Drive/swedracor/TopeliusZ-Perdita.xml', 'r', encoding='utf-8') as f:
    xml = f.read()

In [0]:
# replace soft hyphen by true hyphen
xml = re.sub(r'\xad', '-', xml)

# add mising #
xml = re.sub(r'(<sp who="#spCarl) (spDoktorn">)', r'\1 #\2', xml)

# remove <figure>...</figure>
xml = re.sub(r'<figure(?: [^<>]*?)?>.*?</figure>', r'', xml, 0, re.DOTALL)
xml = re.sub(r'</figure>', r'', xml, 0, re.DOTALL)

# remove some tags
for tag in 'lb placeName emph orgName said surplus writing rs supplied cb performance quote unclear delSpan floatingText actor bibl hi note time name anchor gap pb ref'.split():
    xml = re.sub(fr'</?{tag}(?: [^<>]*?)?/?>', '', xml)

# from '<choice> <sic>X</sic> <corr>Y</corr> </choice>' choose correction (Y)
xml = re.sub(r'<choice>[^<>]*?<sic(?: [^<>]*?)?/?>[^<>]*?(?:</sic>)?[^<>]*?<corr(?: [^<>]*?)?>(.*?)</corr>[^<>]*?</choice>', r'\1', xml, 0, re.DOTALL)

# from '<choice> <orig>X</orig> <reg>Y</reg> </choice>' choose correction (Y)
xml = re.sub(r'<choice>[^<>]*?<orig>.*?</orig>[^<>]*?<reg>(.*?)</reg>[^<>]*?</choice>', r'\1', xml, 0, re.DOTALL)

# remove <forename type="alt">Agust</forename>
# remove <surname type="alt">Green</surname>
for alt_name in ['forename', 'surname']:
    xml = re.sub(fr'<{alt_name} type="alt">[^<>]*?</{alt_name}>', r'', xml, 0, re.DOTALL)

for outer_tag in ['body', 'epilogue', 'set', 'roleDesc']:
    try:
        old_text = re.findall(fr'(<{outer_tag}[^<>]*?>.*?</{outer_tag}>)', xml, re.DOTALL)[0]
    except IndexError:
        continue

    text = old_text

    for tag in 'persName title role'.split(): # date
        text = re.sub(fr'</?{tag}(?: [^<>]*?)?/?>', '', text)

    xml = re.sub(re.escape(old_text), text, xml, re.DOTALL)

old_xml = xml

Define schema:
schema = 'http://www.tei-c.org/ns/1.0'
schema_with_brackets = '{' + schema + '}

Let's replace "\<idno type="dracor">swe000044\</tei:idno>" with "\<idno type="dracor">swe000044\</idno>" (so that etree could parse the file)

In [0]:
xml = re.sub(r'(<idno type="dracor">)([^<>]*?)</tei:idno>', r'\1\2</idno>', old_xml)

We will remove the comments as we don't save such information

In [0]:
parser = etree.XMLParser(remove_comments=True, remove_blank_text=True)
root = etree.fromstring(xml, parser=parser)

In [0]:
# strip all namespaces
for tag in root.iter():
    if '}' in tag.tag:
        tag.tag = tag.tag.split('}', 1)[1]

removing anchors/gaps/pbs (?)

In [0]:
# for tag in 'anchor gap pb'.split():
#     for anchor in root.xpath('//' + tag):
#         parent = anchor.getparent()
#         parent.remove(anchor)

Define prefix

In [0]:
prefix = '{http://www.w3.org/XML/1998/namespace}'

Let's add 'xml-stylesheet' tag and set lang in TEI tag:

In [0]:
root.addprevious(etree.PI('xml-stylesheet', 'type="text/css" href="../css/tei.css"'))
root.set(prefix + "lang", "sv")

In [0]:
teiHeader, text = root.getchildren()

In [0]:
fileDesc, encodingDesc, profileDesc = teiHeader.getchildren()

In [0]:
try:
    titleStmt, publicationStmt, sourceDesc = fileDesc.getchildren()
except ValueError:
    titleStmt, editionStmt, publicationStmt, sourceDesc = fileDesc.getchildren()
    fileDesc.remove(editionStmt)

In [7116]:
titleStmt_children = titleStmt.getchildren()

sub_title = None
if titleStmt_children[1].tag == 'title':
    # there is a sub-title
    if titleStmt_children[2].tag == 'title':
        main_title, sub_title, sub_title_two, author, *others = titleStmt_children
        sub_title.text += ' (' + sub_title_two.text + ')'
    else:
        main_title, sub_title, author, *others = titleStmt_children
else:
    # there is no sub-title
    main_title, author, *others = titleStmt_children
if main_title.text is None:
    main_title.text = ''
    for part_of_text in main_title.itertext():
        main_title.text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
    main_title.text = main_title.text.strip()
    for child in main_title.getchildren():
        main_title.remove(child)
    main_title.text = main_title.text.strip()
    print(main_title.text)

Perdita eller Den förlorade våren


Let's add xml:lang attribute to the title(s):

In [7117]:
main_title

<Element title at 0x7f4be2c85bc8>

In [0]:
main_title.set(prefix + "lang", "sv")
main_title.text = main_title.text.strip('"')
if sub_title is not None:
    sub_title.set(prefix + "lang", "sv")

Let's add translations (English) of titles to the 'titleStmt':

In [0]:
main_title_en = etree.Element('title', type='main')

# set xml:lang
main_title_en.set(prefix + "lang", "en")

en_translation = translator.translate(main_title.text, src='sv', dest='en').text
# capitalize non-functional words
main_title_en.text = titlecase(en_translation).strip().strip('"')
# insert after main_title on swedish
titleStmt.insert(1, main_title_en)

In [0]:
if sub_title is not None:
    sub_title_en = etree.Element('title', type='sub')

    # set xml:lang
    sub_title_en.set(prefix + "lang", "en")

    en_translation = translator.translate(sub_title.text, src='sv', dest='en').text
    # capitalize non-functional words
    sub_title_en.text = titlecase(en_translation).strip()
    # insert after sub_title on swedish
    titleStmt.insert(3, sub_title_en)

In [0]:
forename, nameLink, surname = [None] * 3
for child in author.getchildren()[0].getchildren():
    if child.tag == 'forename':
        forename = child.text
    elif child.tag == 'nameLink':
        nameLink = child.text
    elif child.tag == 'surname':
        surname = child.text

In [0]:
name = ''
if forename is not None:
    name += forename + ' '
if nameLink is not None:
    name += nameLink + ' '
if surname is not None:
    name += surname
name = name.strip()
if not name:
    name = author.getchildren()[0].text
    forename, surname = name.split()

In [0]:
# with open('authors_to_wikidata_urls.json', 'r', encoding='utf-8') as f:
#     authors_to_wikidata_urls = json.load(f)

authors_to_wikidata_urls = {}

with open('/content/drive/My Drive/SweDraCor_authors_Wikidata_ids.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None) # skip header
    for row in reader:
        authors_to_wikidata_urls[row[0]] = row[1]

with open('authors_to_wikidata_urls.json', 'w', encoding='utf-8') as f:
    json.dump(authors_to_wikidata_urls, f)

In [0]:
# with open('plays_to_wikidata_urls.json', 'r', encoding='utf-8') as f:
#     plays_to_wikidata_urls = json.load(f)

plays_to_wikidata_urls = {}

with open('/content/drive/My Drive/SweDraCor_plays_Wikidata_ids.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None) # skip header
    for row in reader:
        plays_to_wikidata_urls[row[1]] = row[3]

with open('plays_to_wikidata_urls.json', 'w', encoding='utf-8') as f:
    json.dump(plays_to_wikidata_urls, f)

In [0]:
# remove old author
titleStmt.remove(author)

# add new author
author = etree.Element('author', key='wikidata:' + authors_to_wikidata_urls[name].split('/')[-1])
if nameLink is not None:
    author.text = nameLink + ' ' + surname + ', ' + forename
elif forename is not None and surname is not None:
    author.text = surname + ', ' + forename
titleStmt.append(author)

Let's delete reference information (sponsors/funders/editors/principals/encoders):

In [0]:
for other in others:
    titleStmt.remove(other)

Let's remove editionStmt:

In [0]:
try:
    publisher, address, first_idno, second_idno, availability = publicationStmt.getchildren()
except Exception as e:
    print(e)

In [0]:
publisher.text = 'DraCor'
publisher.set(prefix + "id", "dracor")

Let's remove address:

In [0]:
publicationStmt.remove(address)

In [0]:
first_idno.set('type', 'URL')
first_idno.text = 'https://dracor.org'

In [0]:
second_idno.set(prefix + "base", "https://dracor.org/id/")

In [0]:
del availability.attrib['status']

In [0]:
licence = availability.getchildren()[0]

In [0]:
del licence.attrib['target']

In [0]:
for child in licence.getchildren():
    licence.remove(child)

In [0]:
ab = etree.Element('ab')
ab.text = 'CC0'
licence.append(ab)

In [0]:
ref = etree.Element('ref', target='https://creativecommons.org/publicdomain/zero/1.0/')
ref.text = 'Licence'
licence.append(ref)

In [0]:
idno = etree.Element('idno', type='wikidata')
idno.set(prefix + "base", "https://www.wikidata.org/entity/")
idno.text = plays_to_wikidata_urls[main_title.text.strip('"')].split('/')[-1]

In [0]:
publicationStmt.append(idno)

In [0]:
biblStruct = sourceDesc.getchildren()[0]

In [0]:
sourceDesc.remove(biblStruct)

In [0]:
if biblStruct.tag == 'listBibl':
    biblStruct = biblStruct.getchildren()[0]

In [0]:
biblStruct_children = biblStruct.getchildren()
monogr = None
for biblStruct_child in biblStruct_children:
    if biblStruct_child.tag == 'monogr':
        monogr = biblStruct_child
        break

In [0]:
titles, idno, imprint = [None] * 3
for monogr_child in monogr.getchildren():
    if monogr_child.tag == 'title':
        if titles is None:
            titles = []
        titles.append(monogr_child)
    elif monogr_child.tag == 'idno':
        idno = monogr_child
    elif monogr_child.tag == 'imprint':
        imprint = monogr_child

In [0]:
if isinstance(titles[0], list) and titles[0][-1].tag == 'idno':
    idno = titles[0][-1]
    titles = titles[0][:-1]

In [0]:
url = idno.text
title_text = ''
for title in titles:
    if title.text is None:
        title.text = ''
        for part_of_text in title.itertext():
            title_text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
        title_text.strip()
    title_text += title.text.strip() + ', '
title_text = title_text.strip(', ')

In [0]:
pubPlace, publisher, date, biblScopes, respStmt = [None] * 5
for imprint_child in imprint.getchildren():
    if imprint_child.tag == 'pubPlace':
        pubPlace = imprint_child
    elif imprint_child.tag == 'publisher':
        publisher = imprint_child
    elif imprint_child.tag == 'date':
        date = imprint_child
    elif imprint_child.tag == 'biblScope':
        if biblScopes is None:
            biblScopes = []
        biblScopes.append(imprint_child)
    elif imprint_child.tag == 'respStmt':
        respStmt = imprint_child
    else:
        print(imprint_child)
        raise ValueError("WTF?")

In [0]:
when = None
if date is not None:
    when = date.attrib['when']

In [0]:
bibl = etree.Element('bibl', type='digitalSource')
sourceDesc.append(bibl)

In [0]:
name = etree.Element('name')
name.text = ''
bibl.append(name)

In [0]:
idno = etree.Element('idno', type='URL')
idno.text = url
bibl.append(idno)

In [0]:
availability = etree.Element('availability', status='free')
p = etree.Element('p')
p.text = 'In the public domain.'
availability.append(p)
bibl.append(availability)

In [0]:
inner_bibl = etree.Element('bibl', type='originalSource')
bibl.append(inner_bibl)

In [0]:
if publisher is not None and publisher.text is None:
    publisher.text = ''
    for part_of_text in publisher.itertext():
        publisher.text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
    publisher.text.strip()

In [0]:
title = etree.Element('title')
title.text = title_text + ', '
if pubPlace is not None:
    title.text += pubPlace.text + ', '
if publisher is not None:
    title.text += publisher.text + ', '
if biblScopes is not None:
    for biblScope in biblScopes:
        title.text += biblScope.text + ', '
if respStmt is not None:
    title.text += respStmt.getchildren()[-1].text + ', '
if when is not None:
    title.text += when + ' '
if url is not None:
    title.text += '(' + url + ')'

inner_bibl.append(title)

In [0]:
date = etree.Element('date', type='print', when=when)
date.text = '"' + when + '"' + ' ' + '(' + url + ')'
inner_bibl.append(date)

In [0]:
date = etree.Element('date', type='premiere')
inner_bibl.append(date)

In [0]:
date = etree.Element('date', type='written')
inner_bibl.append(date)

Let's remove encodingDesc

In [0]:
teiHeader.remove(encodingDesc)

profileDesc

In [0]:
for child in profileDesc:
    profileDesc.remove(child)

In [0]:
def add_particDesc_to_profileDesc(profileDesc):
    # particDesc to profileDesc
    particDesc = etree.Element('particDesc')
    profileDesc.append(particDesc)
    return particDesc

In [0]:
def add_listPerson_to_particDesc(particDesc):
    # listPerson to particDesc
    listPerson = etree.Element('listPerson')
    particDesc.append(listPerson)
    return listPerson

In [0]:
particDesc = add_particDesc_to_profileDesc(profileDesc)

In [0]:
listPerson = add_listPerson_to_particDesc(particDesc)

In [0]:
def add_textClass_to_profileDesc(profileDesc):
    # textClass to profileDesc
    textClass = etree.Element('textClass')
    profileDesc.append(textClass)
    return textClass

In [0]:
textClass = add_textClass_to_profileDesc(profileDesc)

In [0]:
def add_keywords_to_textClass(textClass):
    # keywords to textClass
    keywords = etree.Element('keywords')
    textClass.append(keywords)
    return keywords

In [0]:
keywords = add_keywords_to_textClass(textClass)

In [0]:
def add_term_to_keywords(keywords):
    term = etree.Element('term', type='genreTitle', subtype='')
    term.text = ''
    keywords.append(term)
    return term

In [0]:
term = add_term_to_keywords(keywords)

revisionDesc

In [0]:
revisionDesc = etree.Element('revisionDesc')
teiHeader.append(revisionDesc)
listChange = etree.Element('listChange')
revisionDesc.append(listChange)
change = etree.Element('change', when=datetime.datetime.now().strftime("%Y-%m-%d"))
change.text = "(%s) convert from source" % ('eg') # replace if necessary
listChange.append(change)

text

delete xml:id attribute of text

In [0]:
del text.attrib[prefix + 'id']

text children

In [0]:
try:
    front, body, back = text.getchildren()
except ValueError:
    body, back = text.getchildren()

In [0]:
if back.getchildren()[0].tag == 'epilogue':
    body.append(back.getchildren()[0])

In [0]:
try:
    castList = front.getchildren()[-1]
    
    try:
        assert castList.tag == 'castList'
    except AssertionError:
        if castList.tag == 'set':
            body.insert(0, castList)
        castList = front.getchildren()[-1]
        # append castList to body
    body.insert(0, castList)
except IndexError:
    castList = None

In [0]:
if castList is not None:
    castList_children = castList.getchildren()

In [0]:
if castList is not None:
    was_addSpan = 0
    for child_index, child in enumerate(castList_children):
        if child.tag == 'addSpan':
            was_addSpan = 1
            break

In [0]:
if castList is not None:
    # remove addSpan
    if was_addSpan:
        castList.remove(castList_children[child_index])

    # remove castItem type="list"
    while child_index + 1 < len(castList_children) and was_addSpan:
        castList.remove(castList_children[child_index + 1])
        child_index += 1

In [0]:
if castList is not None:
    castList_children = castList.getchildren()
else:
    castList_children = []

remove children of 'front'

In [0]:
for child in front.getchildren():
    front.remove(child)

In [0]:
def add_docTitle_to_front(front):
    # docTitle to front
    docTitle = etree.Element('docTitle')
    front.append(docTitle)
    return docTitle

In [0]:
docTitle = add_docTitle_to_front(front)

In [0]:
def add_titlePart_main_to_docTitle(docTitle, title):
    # titlePart to docTitle
    titlePart = etree.Element('titlePart', type='main')
    titlePart.text = title.strip('"')
    docTitle.append(titlePart)
    return titlePart

In [0]:
def add_titlePart_sub_to_docTitle(docTitle, title):
    # titlePart to docTitle
    titlePart = etree.Element('titlePart', type='sub')
    titlePart.text = title
    docTitle.append(titlePart)
    return titlePart

In [7185]:
add_titlePart_main_to_docTitle(docTitle, main_title.text)

<Element titlePart at 0x7f4be23e37c8>

In [0]:
if sub_title is not None:
    add_titlePart_sub_to_docTitle(docTitle, sub_title.text)

In [0]:
def inner_xml(element):
    return (element.text or '') + ''.join(ET.tostring(e, 'unicode') for e in element)

Change castList:

In [0]:
# id_to_ref = {}
# ref_to_id = {}
first_index = 0
if castList_children and castList_children[0].tag == 'head':
    first_index = 1
for castItem in castList_children[first_index:]:
    roleDesc = ''
    castItem_children = castItem.getchildren()
    if len(castItem_children) == 0:
        continue
    if len(castItem_children) == 2:
        role, roleDesc = castItem_children
    if castItem.tag == 'castGroup':
        for attribute in castItem.attrib:
            del castItem.attrib[attribute]
        *castItems, roleDesc = castItem_children

        for castItem in castItems:
            castItem_children = castItem.getchildren()
            full_text = ''
            for part_of_text in castItem.itertext():
                full_text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '

            for castItem_child in castItem_children:
                castItem.remove(castItem_child)
            castItem.text = re.sub(r'\s{2,}', '', full_text.strip())
        roleDesc.text = roleDesc.text.strip()
        role = None
    else:
        role = castItem_children[0]
#     ref = role.attrib.values()[0][2:]
#     id_ = role.attrib.values()[0]
#     id_to_ref[id_] = ref
#     ref_to_id[ref] = id_
    full_text = ''
    for part_of_text in castItem.itertext():
        full_text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
    if role is not None:
        full_text = ''
        for part_of_text in role.itertext():
            full_text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
        for castItem_child in castItem_children:
            castItem.remove(castItem_child)
    castItem.text = re.sub(r'\s{2,}', '', full_text.strip())

In [0]:
def replace_func(string):
    for letter_with_umlaut, replacement in zip('äåöéè', 'aaoee'):
        string = string.replace(letter_with_umlaut, replacement)
    return string

Also we will delete \<lb/> tag because we don't encode it

In [0]:
byline = None
try:
    *divs, byline = body.getchildren()
    assert divs and byline.tag == 'byline'
except (ValueError, AssertionError):
    byline = None
    divs = body.getchildren()

In [0]:
def recursive_clean(div):
    if div.tag == 'div':
        # delete n attrib
        if 'n' in div.attrib:
            del div.attrib['n']
    elif div.tag == 'epilogue':
        if prefix + 'id' in div.attrib:
            del div.attrib[prefix + 'id']
    elif div.tag == 'sp' and 'who' in div.attrib:
        tmp = ['#' + elem[1 + 2 * (elem.startswith('#sp')):] for elem in div.attrib['who'].split()]
        for i, elem in enumerate(tmp):
            tmp[i] = re.sub(r'([A-ZÅÄÖ][a-zåäö]+)(?=[A-ZÅÄÖ])', r'\1 ', elem[1:]) #.lower()
            tmp[i] = '#' + tmp[i].replace(' ', '_')
        div.attrib['who'] = replace_func(' '.join(tmp))
    for tag in div.getchildren():
        full_text = inner_xml(tag)
        for tag_child in tag.getchildren():
            recursive_clean(tag_child)

        if tag.text:
            tag.text = re.sub(r'\s{2,}', ' ', tag.text).strip()
            tag.text = re.sub('\n', ' ', tag.text)
        recursive_clean(tag)
    if div.text:
        div.text = re.sub(r'\s{2,}', ' ', div.text).strip()
        div.text = re.sub('\n', ' ', div.text)

In [0]:
for div in divs:
    recursive_clean(div)

In [0]:
for sp in root.xpath('//sp'):
    if 'who' not in sp.attrib:
        continue
    tmp = ['#' + elem[1 + 2 * (elem.startswith('#sp')):] for elem in sp.attrib['who'].split()]
    for i, elem in enumerate(tmp):
        tmp[i] = re.sub(r'([A-ZÅÄÖ][a-zåäö]+)(?=[A-ZÅÄÖ])', r'\1 ', elem[1:]).lower()
        tmp[i] = '#' + tmp[i].replace(' ', '_')
    sp.attrib['who'] = replace_func(' '.join(tmp))

In [0]:
if byline is not None:
    date = byline.getchildren()[0]
    if re.fullmatch(r'[0-9]{4,4}', date.text) is not None:
        written_date = bibl.getchildren()[-1].getchildren()[-1]
        written_date.set('when', date.text)
        written_date.text = date.text
    body.remove(byline)

In [0]:
tmp_ind = 1
while back.getchildren()[len(back.getchildren()) - tmp_ind].tag != 'div':
    tmp_ind += 1
div = back.getchildren()[len(back.getchildren()) - tmp_ind]

In [0]:
try:
    tmp_listPerson, listOrg, *_ = div.getchildren()
except ValueError:
    tmp_listPerson, = div.getchildren()

if 'type' in tmp_listPerson.attrib:
    tmp_listPerson = div

In [0]:
noncast_listPerson = None
try:
    cast_listPerson, noncast_listPerson, *_ = tmp_listPerson.getchildren()
    if 'type' in noncast_listPerson and noncast_listPerson.attrib['type'] == 'cast':
        raise ValueError
except ValueError:
    cast_listPerson_epilogue, cast_listPerson_main, *_ = tmp_listPerson.getchildren()
    cast_listPerson = etree.Element('listPerson')
    for inner_listPerson in [cast_listPerson_epilogue, cast_listPerson_main]:
        for child in inner_listPerson:
            cast_listPerson.append(child)

In [0]:
list_of_speakers = []
set_of_speakers = set()
for sp in root.xpath('//sp'):
    if 'who' not in sp.attrib:
        continue
    whos = sp.attrib['who'].split()
    for who in whos:
        if who not in set_of_speakers:
            set_of_speakers.add(who)
            list_of_speakers.append(who)

In [7199]:
list_of_speakers

['#maja',
 '#svante',
 '#ekoroster',
 '#eko',
 '#florio',
 '#fjarilen',
 '#slandan',
 '#goken',
 '#skolbarnen']

In [0]:
list_of_speakers = [re.sub(r'sp([A-Z].*)', r'\1', elem[1:]) for elem in list_of_speakers]

In [0]:
xml_id_to_sex = {}
xml_id_to_fullname = {}
xml_id_to_type_of_person = {}
for type_listPerson in [cast_listPerson, noncast_listPerson]:
    for child in type_listPerson.getchildren():
        if child.tag in ['person', 'personGrp', 'listPerson']:
            if child.tag == 'listPerson':
                childs = child.getchildren()
            else:
                childs = [child]
            for child in childs:
                if child.tag == 'listPerson':
                    inner_children = child.getchildren()
                else:
                    inner_children = [child]
                for inner_child in inner_children:
                    if prefix + 'id' not in inner_child.attrib:
                        continue
                    xml_id = inner_child.attrib[prefix + 'id']
                    xml_id = re.sub(r'([A-ZÅÄÖ][a-zåäö]+)(?=[A-ZÅÄÖ])', r'\1 ', xml_id).lower()
                    xml_id = xml_id.replace(' ', '_')
                    xml_id = replace_func(xml_id)
                    if xml_id in list_of_speakers:
                        if inner_child.tag == 'person':
                            xml_id_to_type_of_person[xml_id] = 'person'
                        else:
                            xml_id_to_type_of_person[xml_id] = 'personGrp'
                        if 'sex' not in inner_child.attrib:
                            xml_id_to_sex[xml_id] = 'UNKNOWN'
                        else:
                            xml_id_to_sex[xml_id] = inner_child.attrib['sex'].upper()
                        try:
                            persName, *_ = inner_child.getchildren()
                        except ValueError:
                            xml_id_to_fullname[xml_id] = ' '.join([elem.capitalize() for elem in xml_id.split('_')])
                            continue
                        full_text = ''
                        for part_of_text in persName.itertext():
                            full_text += re.sub(r'\s{2,}', ' ', part_of_text).strip() + ' '
            #             full_text = inner_xml(persName)
                        full_text = full_text.strip()
                        xml_id_to_fullname[xml_id] = re.sub(r'\s{2,}', r' ', full_text)

In [7202]:
set(list_of_speakers)

{'eko',
 'ekoroster',
 'fjarilen',
 'florio',
 'goken',
 'maja',
 'skolbarnen',
 'slandan',
 'svante'}

In [7203]:
set(list_of_speakers) ^ set(xml_id_to_sex.keys())

set()

In [7204]:
set(list_of_speakers) ^ set(xml_id_to_fullname.keys())

set()

In [7205]:
set(list_of_speakers) ^ set(xml_id_to_type_of_person.keys())

set()

In [0]:
for xml_id in list_of_speakers:
    if xml_id_to_type_of_person[xml_id] == 'person':
        person = etree.Element('person')
        person.set(prefix + "id", xml_id)
        person.set('sex', xml_id_to_sex[xml_id])
        persName = etree.Element('persName')
        persName.text = xml_id_to_fullname[xml_id]
        person.append(persName)
        listPerson.append(person)
    else:
        personGrp = etree.Element('personGrp')
        personGrp.set(prefix + "id", xml_id)
        personGrp.set('sex', xml_id_to_sex[xml_id])
        name = etree.Element('name')
        name.text = xml_id_to_fullname[xml_id]
        personGrp.append(name)
        listPerson.append(personGrp)

In [0]:
text.remove(back)

In [0]:
for stage in text.xpath('//stage'):
    if 'who' in stage.attrib:
        stage.attrib['who'] = replace_func(stage.attrib['who'].lower())

In [0]:
for castList in root.xpath('//castList'):
    if prefix + 'id' in castList.attrib:
        del castList.attrib[prefix + 'id']
    castList_children = castList.getchildren()
    was_addSpan = 0
    for child_index, child in enumerate(castList_children):
        if child.tag == 'addSpan':
            was_addSpan = 1
            break
    # remove addSpan
    if was_addSpan:
        castList.remove(castList_children[child_index])

    # remove castItem type="list"
    if child_index + 1 < len(castList_children) and was_addSpan:
        castList.remove(castList_children[child_index + 1])

In [0]:
for castItem in root.xpath('//castItem'):
    for attribute in castItem.attrib:
        del castItem.attrib[attribute]

In [0]:
for outer_elem in root.xpath('//body//persName'):
    for inner_elem in outer_elem.iter():
        parent = inner_elem.getparent()
        if parent is not None:
            parent.remove(inner_elem)

In [0]:
for tag in ['addSpan', 'role']: # , 'roleDesc'
    for addSpan in root.xpath('//' + tag):
        parent = addSpan.getparent()
        parent.remove(addSpan)

In [0]:
if sub_title is not None:
    if sub_title_en.text.split()[0].lower() in ['comedy', 'drama', 'tragedy']:
        term.attrib['subtype'] = sub_title_en.text.split()[0].lower()
        term.text = '"' + sub_title.text + '"'

In [0]:
for label in text.xpath('//label'):
    for attrib in label.attrib:
        del label.attrib[attrib]

In [7215]:
result = etree.tostring(root.getroottree(), pretty_print=True, xml_declaration=True, encoding="UTF-8").decode('utf-8')
result = html_.unescape(result)
result = re.sub(r'<fw type="pageNum">([0-9]+)</fw>', r'<pb n="\1"/>', result)
result = re.sub(r'([^\s])(<[^/])', r'\1 \2', result)
for i in range(10):
#     print(i)
    result = re.sub(r'(<p(?: rend=\"[^><]*?\")?>(?:(?!</p>).)*?)\s{2,}((?:(?!</p>).)*?</p>)', r'\1 \2', result, 0, re.DOTALL)

result = re.sub(r'(</[^>]+>) (</[^>]+>)', r'\1\2', result)
result = re.sub(r'/>([^\s])', r'/> \1', result)
result = re.sub(r'\s+</stage>', r'</stage>', result)
result = re.sub(r'\( <', r'(<', result)
# result = re.sub(r'> <', r'><', result)
result = re.sub(r',([^\s])', r', \1', result)
result = re.sub(r' ,', r',', result)
result = re.sub(r'\s+([^<\s])', r' \1', result)
result = re.sub(r' (</)', r'\1', result)
result = re.sub(r'(<[^/][^<>]*?>) <', r'\1<', result)
result = re.sub(r' \.', '.', result)
result = re.sub(r'- (<pb [^<>]*?/>) ', r'-\1', result)

result = re.sub(r'\n\s+([^><]*?)</stage>', r' \1</stage>', result)
print(result)

<?xml version='1.0' encoding='UTF-8'?>
<?xml-stylesheet type="text/css" href="../css/tei.css"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:lang="sv">
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title type="main" xml:lang="sv">Perdita eller Den förlorade våren</title>
        <title type="main" xml:lang="en">Perdita or the Lost Spring</title>
        <author key="wikidata:Q139549">Topelius, Zacharias</author>
     </titleStmt>
      <publicationStmt>
        <publisher xml:id="dracor">DraCor</publisher>
        <idno type="URL">https://dracor.org</idno>
        <idno type="dracor" xml:base="https://dracor.org/id/">swe000020</idno>
        <availability>
          <licence>
            <ab>CC0</ab>
            <ref target="https://creativecommons.org/publicdomain/zero/1.0/">Licence</ref>
         </licence>
       </availability>
        <idno type="wikidata" xml:base="https://www.wikidata.org/entity/">Q59652213</idno>
     </publicationStmt>
      <sourceDesc>
        <bib

In [0]:
elemList = set()
for elem in body.iter():
    elemList.add(elem.tag)

In [7217]:
elemList

{'body',
 'castItem',
 'castList',
 'div',
 'fw',
 'head',
 'l',
 'label',
 'lg',
 'p',
 'sp',
 'speaker',
 'stage'}

In [0]:
 # parser =  etree.XMLParser(remove_comments=True, remove_blank_text=True)
# root = etree.fromstring("""<sp who="#spGertrud">
#              <speaker>Gertrud.</speaker>
#              <p>Ja, inte sant! Jag tror att jag aldrig sett en
#                så vacker kläd— — <stage>(hejdar sig, stampar i golfvet).</stage>
#                         </p>
#            </sp>""", parser=parser)

In [0]:
with open(replace_func(surname.lower() + '-' + '-'.join(main_title.text.lower().split()).split('/')[-1]) + '.xml', 'w', encoding='utf-8') as f:
    f.write(result)

In [0]:
# везде: <name></name>

NB:

1) TopeliusZ-Askungen.xml

2) StrindbergA-TillDamaskus.xml
